# 銀行取引最適化ワークフロー

このNotebookは振込手数料と資金移動コストを最小化する最適化ワークフローを提供します。

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, HTML
from datetime import datetime, timedelta

# 自作モジュール
import data_load
import safety
import fee
import optimise
import export
import charts
import monitor
import kpi_logger

## 1. パラメータ設定

最適化のパラメータをスライダーで調整できます。

In [ ]:
# パラメータ設定用ウィジェット
horizon_slider = widgets.IntSlider(
    value=30,
    min=7,
    max=90,
    step=1,
    description='Horizon (days):',
    style={'description_width': 'initial'}
)

quantile_slider = widgets.FloatSlider(
    value=0.95,
    min=0.8,
    max=0.99,
    step=0.01,
    description='Quantile:',
    style={'description_width': 'initial'}
)

lambda_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description='Lambda (penalty):',
    style={'description_width': 'initial'}
)

cutoff_toggle = widgets.Checkbox(
    value=True,
    description='Cut-off time constraints',
    style={'description_width': 'initial'}
)

# ウィジェットを表示
params_box = widgets.VBox([
    widgets.HTML("<h3>最適化パラメータ</h3>"),
    horizon_slider,
    quantile_slider,
    lambda_slider,
    cutoff_toggle
])

display(params_box)

## 2. データ読み込み

必要なCSVファイルを読み込みます。

In [ ]:
# データファイルパス（適宜変更してください）
DATA_DIR = Path("data")

# ファイルパス入力ウィジェット
bank_master_path = widgets.Text(
    value=str(DATA_DIR / "bank_master.csv"),
    description='Bank Master:',
    style={'description_width': 'initial'}
)

fee_table_path = widgets.Text(
    value=str(DATA_DIR / "fee_table.csv"),
    description='Fee Table:',
    style={'description_width': 'initial'}
)

balance_path = widgets.Text(
    value=str(DATA_DIR / "balance_snapshot.csv"),
    description='Balance Snapshot:',
    style={'description_width': 'initial'}
)

cashflow_path = widgets.Text(
    value=str(DATA_DIR / "cashflow_history.csv"),
    description='Cashflow History:',
    style={'description_width': 'initial'}
)

file_paths_box = widgets.VBox([
    widgets.HTML("<h3>データファイルパス</h3>"),
    bank_master_path,
    fee_table_path,
    balance_path,
    cashflow_path
])

display(file_paths_box)

In [ ]:
# データ読み込み関数
def load_data():
    """すべてのCSVファイルを読み込む"""
    try:
        df_bank_master = data_load.load_bank_master(bank_master_path.value)
        df_fee_table = data_load.load_fee_table(fee_table_path.value)
        df_balance = data_load.load_balance(balance_path.value)
        df_cashflow = data_load.load_cashflow(cashflow_path.value)
        
        print("✅ データ読み込み完了")
        print(f"Bank Master: {len(df_bank_master)} 行")
        print(f"Fee Table: {len(df_fee_table)} 行")
        print(f"Balance: {len(df_balance)} 行")
        print(f"Cashflow: {len(df_cashflow)} 行")
        
        return df_bank_master, df_fee_table, df_balance, df_cashflow
        
    except Exception as e:
        print(f"❌ データ読み込みエラー: {e}")
        return None, None, None, None

# 読み込みボタン
load_button = widgets.Button(description="データ読み込み", button_style='primary')
load_output = widgets.Output()

def on_load_clicked(b):
    with load_output:
        load_output.clear_output()
        global df_bank_master, df_fee_table, df_balance, df_cashflow
        df_bank_master, df_fee_table, df_balance, df_cashflow = load_data()

load_button.on_click(on_load_clicked)
display(load_button, load_output)

## 3. 最適化実行

パラメータに基づいて最適化を実行します。

In [ ]:
def run_optimization():
    """最適化を実行する"""
    try:
        # パラメータ取得
        horizon = horizon_slider.value
        quantile = quantile_slider.value
        lambda_penalty = lambda_slider.value
        use_cutoff = cutoff_toggle.value
        
        print(f"🔧 パラメータ: horizon={horizon}, quantile={quantile}, lambda={lambda_penalty}, cutoff={use_cutoff}")
        
        # Safety Stock計算
        with monitor.Timer("Safety Stock計算"):
            safety_stocks = safety.calc_safety(df_cashflow, horizon, quantile)
        print(f"📊 Safety Stock計算完了: {len(safety_stocks)} 銀行")
        
        # 手数料計算器準備
        fee_calc = fee.FeeCalculator(df_fee_table)
        fee_lookup = fee.build_fee_lookup(df_fee_table)
        
        # 初期残高準備
        initial_balance = dict(zip(df_balance['bank_id'], df_balance['balance']))
        safety_dict = safety_stocks.to_dict()
        
        # 銀行・支店情報準備
        banks = df_bank_master['bank_id'].unique().tolist()
        branches = {}
        for bank in banks:
            bank_branches = df_bank_master[df_bank_master['bank_id'] == bank]['branch_id'].unique().tolist()
            branches[bank] = bank_branches
        
        services = df_bank_master['service_id'].unique().tolist()
        
        # 日付範囲（今後30日）
        today = datetime.now()
        days = [(today + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(30)]
        
        # Cut-off時刻制約
        cut_off = None
        if use_cutoff:
            cut_off = {}
            for _, row in df_bank_master.iterrows():
                cut_off[(row['bank_id'], row['service_id'])] = row['cut_off_time']
        
        # ダミーのnet_cash（実際は予測値を使用）
        net_cash = {(bank, day): 0 for bank in banks for day in days}
        
        # 最適化実行
        with monitor.Timer("MILP最適化"):
            result = optimise.build_model(
                banks=banks,
                branches=branches,
                days=days,
                services=services,
                net_cash=net_cash,
                initial_balance=initial_balance,
                safety=safety_dict,
                fee_lookup=fee_lookup,
                cut_off=cut_off,
                lambda_penalty=lambda_penalty
            )
        
        transfers = result['transfers']
        balances = result['balance']
        
        print(f"✅ 最適化完了: {len(transfers)} 件の資金移動")
        
        # 結果をグローバル変数に保存
        global optimization_result
        optimization_result = {
            'transfers': transfers,
            'balances': balances,
            'safety_stocks': safety_stocks,
            'parameters': {
                'horizon': horizon,
                'quantile': quantile,
                'lambda': lambda_penalty,
                'cutoff': use_cutoff
            }
        }
        
        return True
        
    except Exception as e:
        print(f"❌ 最適化エラー: {e}")
        return False

# 最適化実行ボタン
optimize_button = widgets.Button(description="最適化実行", button_style='success')
optimize_output = widgets.Output()

def on_optimize_clicked(b):
    with optimize_output:
        optimize_output.clear_output()
        if 'df_bank_master' not in globals():
            print("⚠️ 先にデータを読み込んでください")
            return
        run_optimization()

optimize_button.on_click(on_optimize_clicked)
display(optimize_button, optimize_output)

## 4. 結果表示・エクスポート

In [ ]:
def display_results():
    """最適化結果を表示する"""
    if 'optimization_result' not in globals():
        print("⚠️ 先に最適化を実行してください")
        return
    
    result = optimization_result
    transfers = result['transfers']
    
    # 転送計画をDataFrameに変換
    transfer_records = []
    total_fee = 0
    
    for (from_bank, from_branch, to_bank, to_branch, service, day), amount in transfers.items():
        if amount > 0:
            try:
                expected_fee = fee.FeeCalculator(df_fee_table).get_fee(
                    from_bank, from_branch, service, int(amount), to_bank, to_branch
                )
            except:
                expected_fee = 0
            
            transfer_records.append({
                'execute_date': day,
                'from_bank': from_bank,
                'from_branch': from_branch,
                'to_bank': to_bank,
                'to_branch': to_branch,
                'service_id': service,
                'amount': int(amount),
                'expected_fee': expected_fee
            })
            total_fee += expected_fee
    
    df_transfers = pd.DataFrame(transfer_records)
    
    print(f"📋 資金移動計画: {len(df_transfers)} 件")
    print(f"💰 総手数料: ¥{total_fee:,}")
    
    if len(df_transfers) > 0:
        display(df_transfers.head(10))
    
    return df_transfers, total_fee

# 結果表示ボタン
results_button = widgets.Button(description="結果表示", button_style='info')
results_output = widgets.Output()

def on_results_clicked(b):
    with results_output:
        results_output.clear_output()
        global df_transfers, total_fee
        df_transfers, total_fee = display_results()

results_button.on_click(on_results_clicked)
display(results_button, results_output)

In [ ]:
# エクスポート機能
output_path = widgets.Text(
    value="output/transfer_plan.csv",
    description='Output Path:',
    style={'description_width': 'initial'}
)

def export_results():
    """結果をCSVとして出力する"""
    if 'df_transfers' not in globals():
        print("⚠️ 先に結果を表示してください")
        return
    
    try:
        # ディレクトリ作成
        output_file = Path(output_path.value)
        output_file.parent.mkdir(parents=True, exist_ok=True)
        
        # CSV出力
        df_transfers.to_csv(output_file, index=False)
        print(f"✅ ファイル出力完了: {output_file}")
        
        # チャート作成
        baseline_cost = total_fee * 1.2  # ダミーのベースライン
        chart_path = output_file.parent / "cost_comparison.png"
        charts.plot_cost_comparison(baseline_cost, total_fee, str(chart_path))
        print(f"📊 チャート出力: {chart_path}")
        
        # KPIログ
        kpi_record = kpi_logger.KPIRecord(
            timestamp=datetime.now(),
            total_fee=total_fee,
            total_shortfall=0,  # 計算省略
            runtime_sec=30.0    # ダミー値
        )
        kpi_logger.append_kpi(kpi_record)
        print("📝 KPIログ記録完了")
        
    except Exception as e:
        print(f"❌ エクスポートエラー: {e}")

export_button = widgets.Button(description="CSV・チャート出力", button_style='warning')
export_output = widgets.Output()

def on_export_clicked(b):
    with export_output:
        export_output.clear_output()
        export_results()

export_button.on_click(on_export_clicked)

export_box = widgets.VBox([
    widgets.HTML("<h3>結果エクスポート</h3>"),
    output_path,
    export_button
])

display(export_box, export_output)

## 5. クイック実行

全工程を一括実行するボタンです。

In [ ]:
def run_full_pipeline():
    """全パイプラインを一括実行"""
    print("🚀 全パイプライン実行開始...")
    
    # 1. データ読み込み
    print("\n1️⃣ データ読み込み...")
    global df_bank_master, df_fee_table, df_balance, df_cashflow
    df_bank_master, df_fee_table, df_balance, df_cashflow = load_data()
    if df_bank_master is None:
        return
    
    # 2. 最適化実行
    print("\n2️⃣ 最適化実行...")
    success = run_optimization()
    if not success:
        return
    
    # 3. 結果表示
    print("\n3️⃣ 結果表示...")
    global df_transfers, total_fee
    df_transfers, total_fee = display_results()
    
    # 4. エクスポート
    print("\n4️⃣ エクスポート...")
    export_results()
    
    print("\n🎉 全パイプライン完了!")

pipeline_button = widgets.Button(
    description="🚀 全実行", 
    button_style='danger',
    layout=widgets.Layout(width='200px', height='50px')
)
pipeline_output = widgets.Output()

def on_pipeline_clicked(b):
    with pipeline_output:
        pipeline_output.clear_output()
        run_full_pipeline()

pipeline_button.on_click(on_pipeline_clicked)
display(pipeline_button, pipeline_output)

---

## 使用方法

1. **パラメータ調整**: 上部のスライダーで最適化パラメータを設定
2. **データ準備**: CSVファイルパスを確認し「データ読み込み」実行
3. **最適化**: 「最適化実行」ボタンでMILP求解
4. **結果確認**: 「結果表示」で転送計画を確認
5. **出力**: 「CSV・チャート出力」で結果保存

**クイック実行**: 🚀ボタンで全工程を一括実行可能

---
*© 2025 Mizuho Digital Planning Team*